# Start writing a tfx pipeline

1. Data ingestion from fifty one 
    a. Convert to a tfrecord
    b. ExampleGen pipeline object 
2. Data Validation? 
3. TFT
4. Training 
5. Analysis and Validatin 

2. and 5. can be skipped to start. how do we actually locally run? 
- wrap with airflow  


In [1]:
from IPython.lib.deepreload import reload
%load_ext autoreload
%autoreload 2

# 1a. encode as tfrecord

In [17]:
filenames = [str(x.filepath) for x in labeled_view.iter_samples()]
filenames[0]
labels = [x.label for x in labeled_view[filenames[0]]["('new_field',)"].classifications]
fergus_labels = 1 if 'fergus' in labels else 0
fergus_labels

1

In [19]:
import fiftyone as fo
from src.tfrecord_encoder import encode_dataset
from pathlib import Path


path = Path("/home/fergus/repos/icloud")
dataset = fo.load_dataset("icloud")
labeled_view = dataset.match({"('new_field',)": {"$exists": True}})
encode_dataset(labeled_view, path)


# 1b. load 

In [20]:
import os

from tfx import v1 as tfx
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext


In [21]:
from tfx.components import ImportExampleGen
from tfx.orchestration.experimental.interactive.interactive_context import \
    InteractiveContext

context = InteractiveContext()
example_gen = ImportExampleGen(input_base="tfrecord")
context.run(example_gen)

ExecutionResult(
    component_id: ImportExampleGen
    execution_id: 1
    outputs:
        examples: OutputChannel(artifact_type=Examples, producer_component_id=ImportExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

In [23]:
from tfx.components import StatisticsGen
statistics_gen = StatisticsGen(examples=example_gen.outputs['examples'])
context.run(statistics_gen)

ExecutionResult(
    component_id: StatisticsGen
    execution_id: 2
    outputs:
        statistics: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=StatisticsGen, output_key=statistics, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

In [29]:
import tensorflow_data_validation as tfdv

stats = tfdv.generate_statistics_from_tfrecord(data_location='tfrecord/icloud_data.tfrecord')

In [30]:
 tfdv.visualize_statistics(stats)

In [33]:
schema = tfdv.infer_schema(stats)

In [39]:
from tfx.components import SchemaGen
schema_gen = SchemaGen(statistics=statistics_gen.outputs['statistics'])

In [40]:
# transform just needs this preprocessing_Fn defined 
from tfx.components import Transform

transform = Transform(
    examples=example_gen.outputs['examples'],
    schema=schema_gen.outputs['schema'],
    module_file=os.path.abspath("tft_module.py"))
context.run(transform)


running bdist_wheel
running build
running build_py
creating build
creating build/lib
copying tft_module.py -> build/lib
installing to /tmp/tmp7rs3vitk
running install
running install_lib
copying build/lib/tft_module.py -> /tmp/tmp7rs3vitk
running install_egg_info
running egg_info
creating tfx_user_code_Transform.egg-info
writing tfx_user_code_Transform.egg-info/PKG-INFO
writing dependency_links to tfx_user_code_Transform.egg-info/dependency_links.txt
writing top-level names to tfx_user_code_Transform.egg-info/top_level.txt
writing manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
reading manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
writing manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
Copying tfx_user_code_Transform.egg-info to /tmp/tmp7rs3vitk/tfx_user_code_Transform-0.0+8570dbb10c47f9fa31e499a00ef898cd2cc4a16e76119a1ef3416f159dde4cf4-py3.9.egg-info
running install_scripts
creating /tmp/tmp7rs3vitk/tfx_user_code_Transform-0.0+8570dbb10c47f9fa3

ValueError: expected list length of one but got 0

In [41]:
import tempfile